In [1]:
# ! python --version

In [4]:
# ! pip install vllm

In [6]:
# ! pip install langchain-community

In [8]:
# ! apt update && apt install -y unzip zip

In [3]:
# ! nvidia-smi

In [9]:
! unzip phase_2_input_data.zip

Archive:  phase_2_input_data.zip
   creating: final_test_data/
  inflating: final_test_data/final_test_data.features  
   creating: training_data/
  inflating: training_data/train.features  
  inflating: training_data/train.labels  


In [11]:
import huggingface_hub
huggingface_hub.login()

In [12]:
from vllm import LLM, SamplingParams

In [13]:
# llm = LLM(model="microsoft/Phi-3.5-mini-instruct", max_model_len=1024)
llm = LLM(model="srinathmkce/indoml_100k_llama_model_16bit", max_model_len=1024)



config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

INFO 10-20 05:39:27 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='srinathmkce/indoml_100k_llama_model_16bit', speculative_config=None, tokenizer='srinathmkce/indoml_100k_llama_model_16bit', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=srinathmkce/indoml_100k_llama_model_16bit, num_scheduler_steps=1, chunked_prefill_en

tokenizer_config.json:   0%|          | 0.00/51.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 10-20 05:39:58 model_runner.py:1067] Loading model weights took 14.9595 GB
INFO 10-20 05:39:58 gpu_executor.py:122] # GPU blocks: 12036, # CPU blocks: 2048
INFO 10-20 05:39:58 gpu_executor.py:126] Maximum concurrency for 1024 tokens per request: 188.06x
INFO 10-20 05:40:00 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-20 05:40:00 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-20 05:40:12 model_runner.py:1523] Graph capturing finished in 12 secs.


In [14]:
sampling_params = SamplingParams(temperature=0.1, max_tokens=128)

In [10]:
# prompts = [
#     "Hello, my name is",
#     "The president of the United States is",
#     "The capital of France is",
#     "The future of AI is",
# ]

# outputs = llm.generate(prompts, sampling_params)

# # Print the outputs.
# for output in outputs:
#     prompt = output.prompt
#     generated_text = output.outputs[0].text
#     print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")


In [15]:
import pandas as pd
import json
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

df = pd.read_json("final_test_data/final_test_data.features", lines=True)

print(df.shape)

random_df = df.sample(100)
random_df.shape

(184664, 4)


(100, 4)

In [16]:
import json
output_list = []
for index, row in tqdm(random_df.iterrows()):

    conversation = [
        {
            "role": "human",
            "content": json.dumps({"description": row["description"], "retailer": row["retailer"], "price": row["price"]})
        }
    ]
    outputs = llm.chat(conversation, sampling_params=sampling_params, use_tqdm=False)
    output_list.append(outputs)
    break

0it [00:01, ?it/s]


In [17]:
def print_outputs(outputs):
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(generated_text)
        # print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
    print("-" * 80)


In [18]:
print_outputs(outputs)

{"supergroup": "biscuits & confectionery & snacks", "group": "snacks", "module": "snacks chips crisps", "brand": "walkers crisps"}
--------------------------------------------------------------------------------


In [20]:
def predict(row):
    conversation = [
        {
            "role": "human",
            "content": json.dumps({"description": row["description"], "retailer": row["retailer"], "price": row["price"]})
        }
    ]
    print(conversation)
    output = llm.chat(conversation, sampling_params=sampling_params, use_tqdm=False)
    return row['indoml_id'], output[0].outputs[0].text


In [25]:
import json

def build_prompt(row):
    # Use default values for missing keys
    description = row.get("description", "unknown")
    retailer = row.get("retailer", "unknown")
    price = row.get("price", "unknown")
    
    try:
        conversation = [
            {
                "role": "human",
                "content": json.dumps({"description": description, "retailer": retailer, "price": price})
            }
        ]
        return conversation
    except Exception as e:
        # If any error occurs during processing, set all values to "unknown"
        return [
            {
                "role": "human",
                "content": json.dumps({"description": "unknown", "retailer": "unknown", "price": "unknown"})
            }
        ]


In [26]:
def predict(prompt):
    batch_output = []
    outputs = llm.chat(prompt, sampling_params=sampling_params, use_tqdm=False)
    for index, output in enumerate(outputs):
        json_str = outputs[index].outputs[0].text
        batch_output.append(json.loads(json_str))
    return batch_output
    

In [86]:
df.shape[0] / 32

5770.75

In [29]:
from tqdm import tqdm
batch_size = 128
batch_conversation = []
result = []
for index, row in tqdm(df.iterrows()):
    prompt = build_prompt(row)
    batch_conversation.append(prompt)
    if len(batch_conversation) == batch_size:
        output = predict(batch_conversation)
        result += output
        # print(len(result))
        batch_conversation=[]

# Process any remaining samples that didn't complete a full batch
if batch_conversation:
    output = predict(batch_conversation)
    result += output



184664it [1:06:06, 46.55it/s]


In [34]:
result_df = pd.DataFrame(result)

In [35]:
result_df.to_csv("./phase_2_test_set3_predictions.csv", index=False)

In [36]:
result_df.columns

Index(['supergroup', 'group', 'module', 'brand', 'retailer'], dtype='object')

In [39]:
result_df = pd.concat([df["indoml_id"], result_df[['supergroup', 'group', 'module', 'brand']]], axis=1)

In [40]:
result_df.isna().sum()

indoml_id     0
supergroup    0
group         0
module        0
brand         0
dtype: int64

In [42]:
result_df.dtypes

indoml_id      int64
supergroup    object
group         object
module        object
brand         object
dtype: object

In [43]:
category_list = []
for index, row in tqdm(result_df.iterrows()):
    # print(row)
    category_dict = {
        "indoml_id": row["indoml_id"],
        "supergroup": row["supergroup"],
        "group": row["group"],
        "module": row["module"],
        "brand": row["brand"]
    }
    category_list.append(category_dict)

184664it [00:07, 23112.54it/s]


In [44]:
# Save predictions to JSON file
import json
with open("test_gradientgurus4.predict", "w") as fp:
    for row in category_list:
        fp.write(json.dumps(row) + "\n")